Installation des librairies requises :

In [1]:
!pip install numpy
!pip install scipy
!pip install matplotlib
!pip install pyqt5


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\albre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\albre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\albre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\albre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Transmission d'un signal binaire pseudo aléatoire :

In [5]:
# -*- coding: utf-8 -*-
#Project: 
'''Développement d'un outil pédagogique de simulation novateur
pour l'enseignement des systèmes de télécommunications numériques'''
#Created on February 2024
#@author: Denis Albrecq & Tom Fontaine
#Version: 1.0
#Last update: 30-04-24



# LIBRAIRIES REQUISES :

import numpy as np
{'''# Importe le module NumPy, utilisé pour les opérations mathématiques'''}

from scipy.signal import butter, lfilter, freqs, lfilter_zi
{'''Importe des fonctions spécifiques du module SciPy pour le traitement des signaux :
- butter : Conception de filtres Butterworth
- lfilter : Filtrage d'un signal avec un filtre linéaire
- freqs : Calcule la réponse en fréquence d'un filtre
- lfilter_zi : Obtient les conditions initiales pour un filtre
- convolve : Effectue une convolution entre deux signaux'''}

import random
{'''Importe le module random pour générer des nombres aléatoires'''}

import matplotlib.pyplot as plt
{'''Importe matplotlib.pyplot pour tracer des graphiques'''}

from matplotlib.widgets import Button, Slider, RadioButtons
{'''Importe des widgets interactifs pour matplotlib :
- Button : Bouton cliquable
- Slider : Curseur interactif
- RadioButtons : Boutons radio pour sélectionner des options'''}

from matplotlib.ticker import MultipleLocator, FuncFormatter
{'''Importe des outils pour formater les axes dans les graphiques matplotlib :
- MultipleLocator : Définit des intervalles réguliers pour les graduations
- FuncFormatter : Permet des formats personnalisés pour les étiquettes des axes'''}

import matplotlib.animation as animation
{'''Importe le module d'animation de matplotlib pour créer des animations'''}

%matplotlib qt
{'''Configurations spéciales pour matplotlib :
Indique à Jupyter d'utiliser l'interface graphique Qt pour afficher les graphiques'''}

from PyQt5.QtGui import QGuiApplication
{'''Importe QGuiApplication, qui fournit les fonctions nécessaires pour une application graphique
basée sur Qt5, principalement pour la gestion des événements de l'interface utilisateur,
des fenêtres, et des aspects graphiques généraux'''}



# LES FONCTIONS :

# Fonction pour générer un signal binaire :
def generate_binary_signal(duration, sampling_rate, bit_rate):
    t = np.arange(0, duration, 1/sampling_rate)
    signal = np.zeros(len(t))
    # À intervalles définis par le ratio sampling_rate/bit_rate, génère un bit aléatoire (0 ou 1)
    # et maintient cette valeur pendant toute la durée du bit
    for i in range(len(t)):
        if i%(int(sampling_rate/bit_rate))==0:
            signal[i]=random.getrandbits(1)
        else :
            signal[i]=signal[i-1]
    return t, signal

# Fonction pour générer un signal binaire à 1! symbole :
def generate_rect_signal(duration, sampling_rate, delta, emplacement) :
    t = np.arange(0, duration, 1/sampling_rate)
    signal = np.zeros(len(t))
    # Si le temps actuel est dans la plage de l'impulsion, définissez le signal à 1
    for i in range(len(t)):
        if t[i]>emplacement*delta and t[i]<(emplacement+1)*delta :
            signal[i]=1
        else :
            signal[i]=0
    return t, signal

# Différents types de code :
def get_code(code_name):
    if code_name == 'NRZ_polaire' :
        return NRZ_polaire
    if code_name == 'NRZ_unipolaire' :
        return NRZ_unipolaire
    if code_name == 'NRZI' :
        return NRZI
    if code_name == 'RZ_polaire' :
        return RZ_polaire
    if code_name == 'RZ_unipolaire' :
        return RZ_unipolaire
    if code_name == 'Manchester' :
        return Manchester
    if code_name == 'Manchester_différentiel' :
        return Manchester_différentiel
    if code_name == 'AMI' :
        return AMI

# Les bits '1' sont mappés à +1 et les bits '0' à -1
def NRZ_polaire(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate
    P_signal = 1
    Covariance = 1
    symbolic_signal=np.zeros(len(binary_signal))
    for i in range(len(binary_signal)):
        symbolic_signal[i]=1 if (binary_signal[i]==1) else -1
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Les bits '1' sont mappés à +1 et les bits '0' restent à 0
def NRZ_unipolaire(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate
    P_signal = 0.5
    Covariance = 0.25
    symbolic_signal=binary_signal
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Un '0' indique un changement de niveau, tandis qu'un '1' maintient le niveau
def NRZI(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate
    P_signal = 1
    Covariance = 1
    symbolic_signal=np.zeros(len(binary_signal))
    symbolic_signal[0]=1 # Commence avec +1 (choix par défaut)
    for i in range(1, len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            if (binary_signal[i]==0):
                symbolic_signal[i]=-1*symbolic_signal[i-1]
            else :
                symbolic_signal[i]=symbolic_signal[i-1]
        else :
            symbolic_signal[i]=symbolic_signal[i-1]

    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Les bits '1' mappés à +1, bits '0' mappés à -1 et retour à zéro au milieu de chaque bit
def RZ_polaire(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate*2
    P_signal = 0.5
    Covariance = 1
    symbolic_signal=np.zeros(len(binary_signal))
    k=0
    for i in range(len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            symbolic_signal[i]=1 if (binary_signal[i]==1) else -1
            k+=1
        elif i%(int(sampling_rate/(2*bit_rate)))==0 :
            symbolic_signal[i]=0
        else :
            symbolic_signal[i]=symbolic_signal[i-1]
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Les bits '1' mappés à +1 et retour à zéro au milieu
def RZ_unipolaire(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate*2
    P_signal = 0.25
    Covariance = 0.25
    symbolic_signal=np.zeros(len(binary_signal))
    k=0
    for i in range(len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            symbolic_signal[i]=binary_signal[i]
            k+=1
        elif i%(int(sampling_rate/(2*bit_rate)))==0 :
            symbolic_signal[i]=0
        else :
            symbolic_signal[i]=symbolic_signal[i-1]
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Chaque bit '1' commence par +1 et retourne à -1, et chaque '0' commence par -1 et retourne à +1
def Manchester(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate*2
    P_signal = 0.5
    Covariance = 1
    symbolic_signal=np.zeros(len(binary_signal))
    k=0
    for i in range(len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            if (binary_signal[i]==0) :
                symbolic_signal[i]=-1
            else :
                symbolic_signal[i]=1
            k+=1
        elif i%(int(sampling_rate/(2*bit_rate)))==0 :
            symbolic_signal[i]=-1*symbolic_signal[i-1]
        else :
            symbolic_signal[i]=symbolic_signal[i-1]
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Un '1' signifie maintien du niveau, un '0' signifie inversion du niveau
def Manchester_différentiel(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate*2
    P_signal = 0.5
    Covariance = 1
    symbolic_signal=np.zeros(len(binary_signal))
    k=0
    symbolic_signal[0]=1 # Commence avec +1 (choix par défaut)
    for i in range(1, len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            if (binary_signal[i]==0) :
                symbolic_signal[i]=-1*symbolic_signal[i-1]
            else :
                symbolic_signal[i]=symbolic_signal[i-1]
            k+=1
        elif i%(int(sampling_rate/(2*bit_rate)))==0 :
            symbolic_signal[i]=-1*symbolic_signal[i-1]
        else :
            symbolic_signal[i]=symbolic_signal[i-1]
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Un '1' change d'un niveau positif à un niveau négatif de façon alternée, et un '0' reste à zéro  
def AMI(binary_signal, bit_rate, sampling_rate):
    symbolic_rate = bit_rate
    P_signal = 1
    Covariance = 0.5
    symbolic_signal=np.zeros(len(binary_signal))
    k=1 # Niveau initial +1 (choix par défaut)
    for i in range(len(binary_signal)):
        if i%(int(sampling_rate/bit_rate))==0:
            if (binary_signal[i]==1):
                symbolic_signal[i]=k
                if k==1:
                    k=-1
                else :
                    k=1
            else :
                symbolic_signal[i]=0
        else :
            symbolic_signal[i]=symbolic_signal[i-1]
    return symbolic_signal, symbolic_rate, P_signal, Covariance

# Fonction pour appliquer un filtre passe-bas :
def apply_lowpass_filter(signal, normal_cutoff, order):
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    zi = lfilter_zi(b, a)
    filtered_signal, _ = lfilter(b, a, signal, zi=zi*signal[0])
    #Pour la réponse indicielle du filtre
    b, a = butter(order, normal_cutoff, btype='low', analog=True)
    w, h = freqs(b, a)
    return filtered_signal, w, h

# Fonction pour générer le bruit AWGN (bruit additif blanc gaussien) pour chaque échantillon :
def get_random_value(sigma):
    mu=0 #car centré sur la valeur de l'amplitude du symbole
    value=random.gauss(mu, sigma)
    (mu, sigma)
    return value

# Fonction pour appliquer du bruit :
def apply_noise(signal, snr, P_signal, Covariance):
    signal_noise=np.zeros(len(signal))
    Résistance = 1 #[Ohm]
    P_noise=P_signal*(10**(-(snr+10*np.log10(Résistance))/20)) #conversion de puissance
    for i in range(len(signal)):
        signal_noise[i]=signal[i]+get_random_value(np.sqrt(Covariance))*P_noise
    return signal_noise

# Fonction pour appliquer le filtre de nyquist :
def apply_nyquist_filter(signal, duration, sampling_rate, symbolic_rate, delta, rolloff_factor) :
    a = np.zeros(int(duration*symbolic_rate))  # a = suite de symboles
    for i in range(len(a)):
        a[i] = signal[int(((sampling_rate*delta)/2)+(sampling_rate*delta)*i)]
    
    # Fonction en cosinus surélevé
    def get_rc(T, rolloff_factor):
        def rc(t, beta):
            return np.sinc(t)*np.cos(np.pi*beta*t)/(1-(2*beta*t)**2)
        return lambda t: rc(t/T, rolloff_factor)
    
    # Fonction pour générer la transmission du signal comme suit : sum( a[k]*g(t-k*delta) )
    def get_signal(g, a):
        t_ = np.arange(-duration, duration, 1/sampling_rate)
        xt = sum(a[k]*g(t_-(k+1)*delta) for k in range(len(a)))
        signal = xt[int((duration + 2*delta)*sampling_rate):int(2*duration*sampling_rate)]
        begin = np.zeros(int(2*delta*sampling_rate))
        return np.concatenate((begin, signal))
    
    g = get_rc(delta, rolloff_factor)
    return  get_signal(g, a)
'''Remarque : 
ici le code vient chercher une valeur au milieu de chaque symbole, 
la multiplie avec g(=filtre rc) et somme le tout.

Ceci a été fait pour simplifier la charge de calcul, il aurait fallu le faire pour chq point.
Ainsi on aurait pris en compte de manière plus significative l'impact du bruit.

(Il est d'ailleurs tout à fait possible de le faire avec un np.convolve() 
et ce qui se fait aussi rapidement en terme de charge de calcul.
Néanmoins, il faut arrivé à décaler g pour que les symboles soient maximisés au moment d'échantillonage.
Càd à la fin du symbole.)

En outre, nous avons remarqué que le changement n'est pas fort significative,
l'approximation faite est donc tout à fait acceptable.
'''

# Fonction pour formatter les graduations des axes :
def formatter(symbolic_rate, char) :
    return FuncFormatter(lambda x, _: f'{int(x/symbolic_rate)}' + char)

# Fonction pour tracer le signal binaire :
def draw_binary_signal(t, signal, ax, title, bit_rate, duration, val):
    a = np.zeros(int(duration*bit_rate))  # a = suite de symboles
    t_ = np.zeros(len(a))
    for i in range(len(a)):
        a[i] = signal[int(((sampling_rate/bit_rate)/2)+(sampling_rate/bit_rate)*i)]
        t_[i] = t[int(((sampling_rate/bit_rate)/2)+(sampling_rate/bit_rate)*i)]
    line = ax.scatter(t_, a, s=20*dsize/bit_rate)
    ax.set_title(title, fontsize=20*dsize)
    ax.set_xlabel('Time [s]', fontsize=15*dsize, loc='right')
    ax.set_xlim(-0.1,duration+0.1)
    ax.set_ylabel('Amplitude', fontsize=15*dsize)
    ax.set_ylim(-val+1,val)
    ax.xaxis.set_minor_locator(MultipleLocator(1/bit_rate))
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax.axhline(1, color='black', linestyle='--', alpha=0.3, linewidth=0.75)
    ax.axhline(0, color='black', linestyle='--', alpha=0.3, linewidth=0.75)
    ax.grid(axis = 'both', linestyle='--', color='grey', linewidth=0.75)
    return line

# Fonction pour tracer un signal temporel :
def draw_signal(t, signal, ax, title, symbolic_rate, duration, val):
    line, = ax.plot(t, signal)
    ax.set_title(title, fontsize=20*dsize)
    ax.set_xlabel('Time [s]', fontsize=15*dsize, loc='right')
    ax.set_xlim(-0.1,duration+0.1)
    ax.set_ylabel('Amplitude', fontsize=15*dsize)
    ax.set_ylim(-val,val)
    ax.xaxis.set_minor_locator(MultipleLocator(1/symbolic_rate))
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax.axhline(1, color='black', linestyle='--', alpha=0.3, linewidth=0.75)
    ax.axhline(0, color='black', linestyle='--', alpha=0.3, linewidth=0.75)
    ax.axhline(-1, color='black', linestyle='--', alpha=0.3, linewidth=0.75)
    ax.grid(axis = 'both', linestyle='--', color='grey', linewidth=0.75)
    return line

# Fonction pour tracer le spectre d'un signal :
def draw_spectre(signal, ax, title, symbolic_rate, sampling_rate, bit_rate):
    ax.xaxis.set_major_formatter(formatter(bit_rate, '/T'))
    ax.xaxis.set_major_locator(MultipleLocator(symbolic_rate))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(40))
    ax.set_xlim(0,10*symbolic_rate)
    for i in range(10):
        ax.axvline(i*symbolic_rate, color='g', linestyle = '--')
    line = ax.magnitude_spectrum(signal, Fs=sampling_rate, scale='dB', color='C1')[2]
    ax.grid(axis ='y', linestyle='--', color='grey', linewidth=0.75)
    ax.set_title(title, fontsize=20*dsize)
    ax.set_xlabel('Frequency [Hz]', fontsize=15*dsize, loc='right')
    ax.set_ylabel('Amplitude [dB]', fontsize=15*dsize)
    return line

# Fonction pour tracer l'oeil de nyquist :
def drawFullEyeDiagram(signal, ax, delta, sampling_rate, title):
    t_eye = np.arange(0,2*delta,1/sampling_rate)
    for i in range(int(duration/(2*delta))-2):
        x_eye = signal[(int(2*sampling_rate*delta*(i+2))):(int(2*sampling_rate*delta*(i+3)))]
        ax.plot(t_eye,x_eye, 'b-')
    ax.set_title(title, fontsize=10)
    ax.set_ylabel('Amplitude', fontsize=8)
    ax.xaxis.set_major_formatter(formatter(delta, chr(8710)))
    ax.xaxis.set_major_locator(MultipleLocator(delta))
    ax.xaxis.set_minor_locator(MultipleLocator(0.1*delta))
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax.grid(axis = 'both', linestyle='--', color='grey', linewidth=0.75)

# Fonction pour afficher les graphes du filtre de nyquist :
def plot_nyquist(duration, symbolic_rate, sampling_rate, delta, rolloff_factor, ax2_5, ax2_6) :
    t_g = np.arange(-duration/symbolic_rate, duration/symbolic_rate, 1/sampling_rate)
    g=np.zeros(len(t_g))
    for i in range(len(t_g)) :
        g[i] = (((np.sinc(t_g[i]/delta)) * np.cos((np.pi*rolloff_factor*t_g[i])/delta)) / ( 1 - ((2*rolloff_factor*t_g[i])/delta)**2))/delta
    line2_5, = ax2_5.plot(t_g, g)
    ax2_5.set_title('Time response', fontsize=20*dsize)
    ax2_5.set_xlabel('Time [s]', fontsize=15*dsize, loc='right')
    ax2_5.set_ylabel('Amplitude', fontsize=15*dsize)
    ax2_5.xaxis.set_minor_locator(MultipleLocator(delta))
    ax2_5.yaxis.set_major_locator(MultipleLocator(0.5/delta))
    ax2_5.grid(axis = 'y', linestyle='--', color='grey', linewidth=0.75)
    
    delta_f=(rolloff_factor*symbolic_rate)/2
    f=np.arange(-2*symbolic_rate, 2*symbolic_rate, 1/sampling_rate)
    g_f=np.zeros(len(f))
    for i in range (int(sampling_rate*symbolic_rate*4)):
        if abs(f[i])<=(symbolic_rate/2)-delta_f :
            g_f[i]=1
        elif abs(f[i])>=(symbolic_rate/2)+delta_f :
            g_f[i]=0
        elif rolloff_factor!=0 :
            g_f[i]=0.5*(1-np.sin(((np.pi*abs(f[i]))/(rolloff_factor*symbolic_rate))-(np.pi/(2*rolloff_factor))))
    ax2_6.xaxis.set_major_formatter(formatter(symbolic_rate, '/T'))
    ax2_6.xaxis.set_major_locator(MultipleLocator(symbolic_rate))
    ax2_6.xaxis.set_minor_locator(MultipleLocator(1))
    ax2_6.yaxis.set_minor_locator(MultipleLocator(50))
    ax2_6.set_xlim(-2*symbolic_rate,2*symbolic_rate)
    for i in range(2):
        ax2_6.axvline(i*symbolic_rate, color='g', linestyle = '--')
        ax2_6.axvline(-i*symbolic_rate, color='g', linestyle = '--')
    ax2_6.grid(axis ='y', linestyle='--', color='grey', linewidth=0.75)
    ax2_6.set_title('Frequency response', fontsize=20*dsize)
    ax2_6.set_xlabel('Frequency [Hz]', fontsize=15*dsize, loc='right')
    ax2_6.set_ylabel('Amplitude', fontsize=15*dsize)
    ax2_6.plot(f, g_f)



# INITIALISATION :

# Paramètres du signal :
duration = 20.0  # durée en secondes
bit_rate = 5 # nombre de bits par seconde
sampling_rate = int(bit_rate*50)  # fréquence d'échantillonage en [Hz]
symbolic_rate = bit_rate # nombre de symboles par seconde

# Création du signal binaire :
t, binary_signal = generate_binary_signal(duration, sampling_rate, bit_rate)

# Codage du signal :
list_code = ('NRZ_polaire','NRZ_unipolaire','NRZI','RZ_polaire','RZ_unipolaire','Manchester','Manchester_différentiel','AMI')
code_name = list_code[0] # Choix par défaut du type de code
code = get_code(code_name)
symbolic_signal, symbolic_rate, P_signal, Covariance = code(binary_signal, bit_rate, sampling_rate)

# Paramètres du canal :
snr = 40 # rapport signal sur bruit en [dB]
# Paramètres filtre passe-bas représentant le canal :
cutoff_frequency = 10 # fréquence de coupure en [Hz] du filtre
normal_cutoff= (2*cutoff_frequency) / sampling_rate 
order = 1 # ordre du filtre

# Faire passer le signal dans le canal :
symbolic_signal_filtered, w, h = apply_lowpass_filter(symbolic_signal, normal_cutoff, order)
symbolic_signal_out = apply_noise(symbolic_signal_filtered, snr, P_signal, Covariance)

# Appliquer le filtre de nyquist :
rolloff_factor = 0.5 # facteur de roll-off
delta = 1/symbolic_rate # durée en secondes d'un symbole
filtered_signal = apply_nyquist_filter(symbolic_signal_out, duration, sampling_rate, symbolic_rate, delta, rolloff_factor)



# FENETRE PRINCIPALE :

# Récupérer l'écran principal :
screen = QGuiApplication.primaryScreen()

# Récupérer la géométrie de l'écran :
screen_geometry = screen.geometry()

# Ajuster la taille de la fenêtre à celle de l'écran :
height_window = screen_geometry.height()/150
width_window = screen_geometry.width()/150
dsize=height_window/width_window

# Créer la fenêtre :
homepage=plt.figure("Transmission of a pseudo-random binary sequence", figsize=(width_window,height_window))
axs_homepage = homepage.subplots(4,2)  # 4 lignes et 2 colonnes

# Créer les graphes :

# Signal binaire
line0_1 = draw_binary_signal(t, binary_signal, axs_homepage[0,0], "Binary signal", bit_rate, duration, val=1.5)

plt.delaxes(axs_homepage[0,1])

# Signal symbolique
line0_3 = draw_signal(t, symbolic_signal, axs_homepage[1,0], "Symbolic signal", symbolic_rate, duration, val=1.5)

# Spectre du signal symbolique
line0_4 = draw_spectre(symbolic_signal, axs_homepage[1,1], "Spectrum of the symbolic signal", symbolic_rate, sampling_rate, bit_rate)

# Signal symbolique de sortie
line0_5 = draw_signal(t, symbolic_signal_out, axs_homepage[2,0], "Symbolic output signal", symbolic_rate, duration, val=1.5)

# Spectre du signal symbolique de sortie
line0_6 = draw_spectre(symbolic_signal_out, axs_homepage[2,1], "Spectrum of the symbolic output signal", symbolic_rate, sampling_rate, bit_rate)

# Signal filtré par nyquist
line0_7 = draw_signal(t, filtered_signal, axs_homepage[3,0], "Nyquist-filtered signal", symbolic_rate, duration, val=2)

# Spectre du signal filtré par nyquist
line0_8 = draw_spectre(filtered_signal, axs_homepage[3,1], "Spectrum of the Nyquist-filtered signal", symbolic_rate, sampling_rate, bit_rate)

# Ajuster les graphes dans la fenêtre :
homepage.subplots_adjust(left=0.25, bottom=0.15,hspace=2*dsize, wspace=dsize/2)



# FENETRE DE L'OEUIL DU SIGNAL EN SORTIE DE CANAL :
oeil_outcanal_window = plt.figure("Eye pattern of the output signal", figsize=(width_window/2, height_window/2))
plt.close()
ax_oeil_outcanal_window = oeil_outcanal_window.add_subplot(111)
oeil_outcanal_window.subplots_adjust(left=0.2, bottom=0.1,hspace=1.5, wspace=0.5)



# FENETRE DE L'OEUIL DE NYQUIST :
oeil_window = plt.figure("Nyquist eye pattern", figsize=(width_window/2, height_window/2))
plt.close()
ax_oeil_window = oeil_window.add_subplot(111)
oeil_window.subplots_adjust(left=0.2, bottom=0.1,hspace=1.5, wspace=0.5)



# FENETRE DE L'INFLUENCE DU CANAL :
canal_window = plt.figure("Influence of the channel", figsize=(2*width_window/3, 2*height_window/3))
plt.close()

# Créer un signal avec 1! symbole :
rect_t, rect_signal=generate_rect_signal(duration/2, sampling_rate, 1/bit_rate, emplacement=1)

# Faire passer le signal dans le canal :
rect_signal_filtered, w, h = apply_lowpass_filter(rect_signal, normal_cutoff, order)
rect_signal_out = apply_noise(rect_signal_filtered, snr, P_signal=1, Covariance=1)

# Créer les graphes :

# Signal d'entrée
ax1_1=canal_window.add_subplot(321)
line1_1 = draw_signal(rect_t, rect_signal, ax1_1, "Input signal", bit_rate, duration/(2+int(bit_rate/2)), val=1.5)

# Spectre du signal d'entrée
ax1_2=canal_window.add_subplot(322)
line1_2 = draw_spectre(rect_signal, ax1_2, "Spectrum of the input signal", bit_rate, sampling_rate, bit_rate)

# Signal de sortie
ax1_3=canal_window.add_subplot(325)
line1_3 = draw_signal(rect_t, rect_signal_out, ax1_3, "Output signal", bit_rate, duration/(2+int(bit_rate/2)), val=1.5)

# Spectre du signal de sortie
ax1_4=canal_window.add_subplot(326)
line1_4 = draw_spectre(rect_signal_out, ax1_4, "Spectrum of the output signal", bit_rate, sampling_rate, bit_rate)

# Réponse en fréquence du canal
ax1_5=canal_window.add_subplot(312)
line1_5 = ax1_5.semilogx((w*sampling_rate)/2, 20 * np.log10(abs(h)))
ax1_5.set_title('Frequency of the canal\n(Butterworth low-pass filter)', fontsize=20*dsize)
ax1_5.set_xlabel('Frequency [Hz]', fontsize=15*dsize, loc='right')
ax1_5.set_ylabel('Gain [dB]', fontsize=15*dsize)
ax1_5.grid(which='both', axis='both')
ax1_5.axvline(cutoff_frequency, color='green')

# Ajuster le graphe dans la fenêtre :
canal_window.subplots_adjust(left=0.3, bottom=0.15,hspace=1.2, wspace=0.4)



# FENETRE DU FILTRE DE NYQUIST :
nyquist_window = plt.figure("Nyquist filter", figsize=(4*width_window/5, 2*height_window/3))
plt.close()

# Créer un signal avec 1! symbole :
rect_t_ny, rect_signal_ny=generate_rect_signal(duration, sampling_rate, 1/symbolic_rate, emplacement=int(symbolic_rate*duration/10)-1)
# Appliquer le filtre de nyquist
rect_signal_ny_filtered = apply_nyquist_filter(rect_signal_ny, duration, sampling_rate, symbolic_rate, delta, rolloff_factor)

# Créer les graphes :

# Signal d'entrée
ax2_1=nyquist_window.add_subplot(321)
line2_1 = draw_signal(rect_t_ny, rect_signal_ny, ax2_1, "Input signal", bit_rate, duration/5, val=1.5)

# Spectre du signal d'entrée
ax2_2=nyquist_window.add_subplot(322)
line2_2 = draw_spectre(rect_signal_ny, ax2_2, "Spectrum of the input signal", symbolic_rate, sampling_rate, bit_rate)

# Signal filtré
ax2_3=nyquist_window.add_subplot(325)
line2_3 = draw_signal(rect_t_ny, rect_signal_ny_filtered, ax2_3, "Filtered signal", bit_rate, duration/5, val=2)

# Spectre du signal filtré
ax2_4=nyquist_window.add_subplot(326)
line2_4 = draw_spectre(rect_signal_ny_filtered, ax2_4, "Spectrum of the filtered signal", symbolic_rate, sampling_rate, bit_rate)

# Réponse temporelle et fréquentielle du filtre de nyquist
ax2_5=nyquist_window.add_subplot(323)
ax2_6=nyquist_window.add_subplot(324)
plot_nyquist(duration, bit_rate, sampling_rate, delta, rolloff_factor, ax2_5, ax2_6)

# Ajuster le graphe dans la fenêtre :
nyquist_window.subplots_adjust(left=0.2, bottom=0.1,hspace=1.2, wspace=0.4)



# BOUTON DE RELOAD :

def regenere(event):
    global t, binary_signal, filtered_signal, rect_t, rect_signal, normal_cutoff, delta, sampling_rate, symbolic_signal, symbolic_signal_out, symbolic_rate
    sampling_rate = int(bit_rate_slider.val*50)
    normal_cutoff = (2*cutoff_frequency_slider.val) / sampling_rate

    # Création d'une autre suite binaire :
    t, binary_signal = generate_binary_signal(duration, sampling_rate, bit_rate_slider.val)
    # Codage du signal :
    code = get_code(code_name)
    symbolic_signal, symbolic_rate, P_signal, Covariance = code(binary_signal, bit_rate_slider.val, sampling_rate)
    delta = 1/symbolic_rate
    # Faire passer le signal dans le canal :
    symbolic_signal_filtered, w, h = apply_lowpass_filter(symbolic_signal, normal_cutoff, order_slider.val)
    symbolic_signal_out = apply_noise(symbolic_signal_filtered, snr_slider.val, P_signal, Covariance)
    # Appliquer le filtre de nyquist :
    filtered_signal = apply_nyquist_filter(symbolic_signal_out, duration, sampling_rate, symbolic_rate, delta, rolloff_factor_slider.val)

    # Retracer les graphes :
    axs_homepage[0,0].cla()
    line0_1 = draw_binary_signal(t, binary_signal, axs_homepage[0,0], "Binary signal", bit_rate_slider.val, duration, val=1.5)
    line0_3.set_data(t, symbolic_signal)
    axs_homepage[1,1].cla()
    line0_4 = draw_spectre(symbolic_signal, axs_homepage[1,1], "Spectrum of the symbolic signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    line0_5.set_data(t, symbolic_signal_out)
    axs_homepage[2,1].cla()
    line0_6 = draw_spectre(symbolic_signal_out, axs_homepage[2,1], "Spectrum of the symbolic output signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    line0_7.set_data(t, filtered_signal)
    axs_homepage[3,1].cla()
    line0_8 = draw_spectre(filtered_signal, axs_homepage[3,1], "Spectrum of the Nyquist-filtered signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    
    # Mettre à jour la fenêtre principale :
    homepage.canvas.draw_idle()
    
    # Mettre à jour la fenêtre du signal en sortie de canal :
    ax_oeil_outcanal_window.cla()
    drawFullEyeDiagram(symbolic_signal_out, ax_oeil_outcanal_window, 1/bit_rate_slider.val, sampling_rate, "Eye pattern of the output signal")
    oeil_outcanal_window.canvas.draw_idle()

    # Mettre à jour la fenêtre de l'oeil de nyquist :
    ax_oeil_window.cla()
    drawFullEyeDiagram(filtered_signal, ax_oeil_window, 1/bit_rate_slider.val, sampling_rate, "Nyquist eye pattern")
    oeil_window.canvas.draw_idle()

    # Mettre à jour le signal d'un seul symbole :
    rect_t, rect_signal = generate_rect_signal(duration/2, sampling_rate, delta, emplacement=1)
    rect_signal_filtered, w, h = apply_lowpass_filter(rect_signal, normal_cutoff, order_slider.val)
    rect_signal_out = apply_noise(rect_signal_filtered, snr_slider.val, P_signal=1, Covariance=1)

    # Retracer les graphes :
    ax1_1.cla()
    line1_1 = draw_signal(rect_t, rect_signal, ax1_1, "Input signal", bit_rate_slider.val, duration/(2+int(bit_rate_slider.val/2)), val=1.5)
    ax1_2.cla()
    line1_2 = draw_spectre(rect_signal, ax1_2, "Spectrum of the input signal", bit_rate_slider.val, sampling_rate, bit_rate_slider.val)
    ax1_3.cla()
    line1_3 = draw_signal(rect_t, rect_signal_out, ax1_3, "Output signal", bit_rate_slider.val, duration/(2+int(bit_rate_slider.val/2)), val=1.5)
    ax1_4.cla()
    line1_4 = draw_spectre(rect_signal_out, ax1_4, "Spectrum of the output signal", bit_rate_slider.val, sampling_rate, bit_rate_slider.val)
    ax1_5.cla()
    line1_5 = ax1_5.semilogx(w*sampling_rate/2, 20 * np.log10(abs(h)))
    ax1_5.set_title('Frequency response of the canal\n(Butterworth low-pass filter)', fontsize=20*dsize)
    ax1_5.set_xlabel('Frequency [rad/s]', fontsize=15*dsize, loc='right')
    ax1_5.set_ylabel('Gain [dB]', fontsize=15*dsize)
    ax1_5.grid(which='both', axis='both')
    ax1_5.axvline(cutoff_frequency_slider.val, color='green')

    # Mettre à jour la fenêtre de l'influence du canal :
    canal_window.canvas.draw_idle()

    # Mettre à jour le signal d'un seul symbole pour le filtre de nyquist :
    rect_t_ny, rect_signal_ny = generate_rect_signal(duration, sampling_rate, 1/symbolic_rate, emplacement=int(symbolic_rate*duration/10)-1)
    rect_signal_ny_filtered = apply_nyquist_filter(rect_signal_ny, duration, sampling_rate, symbolic_rate, delta, rolloff_factor_slider.val)

    # Retracer les graphes :
    ax2_1.cla()
    line2_1 = draw_signal(rect_t_ny, rect_signal_ny, ax2_1, "Input signal", bit_rate_slider.val, duration/5, val=1.5)
    ax2_2.cla()
    line2_2 = draw_spectre(rect_signal_ny, ax2_2, "Spectrum of the intput signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    ax2_3.cla()
    line2_3 = draw_signal(rect_t_ny, rect_signal_ny_filtered, ax2_3, "Filtered signal", bit_rate_slider.val, duration/5, val=1.5)
    ax2_4.cla()
    line2_4 = draw_spectre(rect_signal_ny_filtered, ax2_4, "Spectrum of the filtered signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    ax2_5.cla()
    ax2_6.cla()
    plot_nyquist(duration, bit_rate_slider.val, sampling_rate, delta, rolloff_factor_slider.val, ax2_5, ax2_6)

    # Mettre à jour la fenêtre du filtre de nyquist :
    nyquist_window.canvas.draw_idle()

# [Gauche, Bas, Largeur, Hauteur]
#donné en pourcentage par rapport aux dimension de la fenêtre
reload_ax= homepage.add_axes([0.74, 0.025, 0.1, 0.04])
button_reload = Button(reload_ax, 'Regenere', hovercolor='0.975')
button_reload.on_clicked(regenere)



# BOUTON DE RESTE GLOBAL :

def reset_all(event):
    bit_rate_slider.reset()
    order_slider.reset()
    cutoff_frequency_slider.reset()
    snr_slider.reset()
    rolloff_factor_slider.reset()
reset_all_ax = homepage.add_axes([0.85,0.025,0.1,0.04])
button_reset_all = Button(reset_all_ax, 'Reset', hovercolor='0.975')
button_reset_all.on_clicked(reset_all)



# BOUTON DE MISE A JOUR :

def update(val):
    global filtered_signal, normal_cutoff, delta, sampling_rate, symbolic_signal_out, symbolic_rate, symbolic_signal
    sampling_rate = int(bit_rate_slider.val*50)
    normal_cutoff = (2*cutoff_frequency_slider.val) / sampling_rate

    # Codage du signal :
    code = get_code(code_name)
    symbolic_signal, symbolic_rate, P_signal, Covariance = code(binary_signal, bit_rate_slider.val, sampling_rate)
    delta = 1/symbolic_rate
    # Faire passer le signal dans le canal :
    symbolic_signal_filtered, w, h = apply_lowpass_filter(symbolic_signal, normal_cutoff, order_slider.val)
    symbolic_signal_out = apply_noise(symbolic_signal_filtered, snr_slider.val, P_signal, Covariance)
    # Appliquer le filtre de nyquist :
    filtered_signal = apply_nyquist_filter(symbolic_signal_out, duration, sampling_rate, symbolic_rate, delta, rolloff_factor_slider.val)

    # Retracer les graphes :
    axs_homepage[0,0].cla()
    line0_1 = draw_binary_signal(t, binary_signal, axs_homepage[0,0], "Binary signal", bit_rate_slider.val, duration, val=1.5)
    line0_3.set_data(t, symbolic_signal)
    axs_homepage[1,1].cla()
    line0_4 = draw_spectre(symbolic_signal, axs_homepage[1,1], "Spectrum of the symbolic signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    line0_5.set_data(t, symbolic_signal_out)
    axs_homepage[2,1].cla()
    line0_6 = draw_spectre(symbolic_signal_out, axs_homepage[2,1], "Spectrum of the symbolic output signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    line0_7.set_data(t, filtered_signal)
    axs_homepage[3,1].cla()
    line0_8 = draw_spectre(filtered_signal, axs_homepage[3,1], "Spectrum of the Nyquist-filtered signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    
    # Mettre à jour la fenêtre principale :
    homepage.canvas.draw_idle()
    
    # Mettre à jour la fenêtre du signal en sortie de canal :
    ax_oeil_outcanal_window.cla()
    drawFullEyeDiagram(symbolic_signal_out, ax_oeil_outcanal_window, 1/bit_rate_slider.val, sampling_rate, "Eye pattern of the output signal")
    oeil_outcanal_window.canvas.draw_idle()

    # Mettre à jour la fenêtre de l'oeil de nyquist :
    ax_oeil_window.cla()
    drawFullEyeDiagram(filtered_signal, ax_oeil_window, 1/bit_rate_slider.val, sampling_rate, "Nyquist eye pattern")
    oeil_window.canvas.draw_idle()

    # Mettre à jour le signal d'un seul symbole :
    rect_t, rect_signal = generate_rect_signal(duration/2, sampling_rate, delta, emplacement=1)
    rect_signal_filtered, w, h = apply_lowpass_filter(rect_signal, normal_cutoff, order_slider.val)
    rect_signal_out = apply_noise(rect_signal_filtered, snr_slider.val, P_signal=1, Covariance=1)

    # Retracer les graphes :
    ax1_1.cla()
    line1_1 = draw_signal(rect_t, rect_signal, ax1_1, "Input signal", bit_rate_slider.val, duration/(2+int(bit_rate_slider.val/2)), val=1.5)
    ax1_2.cla()
    line1_2 = draw_spectre(rect_signal, ax1_2, "Spectrum of the input signal", bit_rate_slider.val, sampling_rate, bit_rate_slider.val)
    ax1_3.cla()
    line1_3 = draw_signal(rect_t, rect_signal_out, ax1_3, "Output signal", bit_rate_slider.val, duration/(2+int(bit_rate_slider.val/2)), val=1.5)
    ax1_4.cla()
    line1_4 = draw_spectre(rect_signal_out, ax1_4, "Spectrum of the output signal", bit_rate_slider.val, sampling_rate, bit_rate_slider.val)
    ax1_5.cla()
    line1_5 = ax1_5.semilogx(w*sampling_rate/2, 20 * np.log10(abs(h)))
    ax1_5.set_title('Frequency response of the canal\n(Butterworth low-pass filter)', fontsize=20*dsize)
    ax1_5.set_xlabel('Frequency [rad/s]', fontsize=15*dsize, loc='right')
    ax1_5.set_ylabel('Gain [dB]', fontsize=15*dsize)
    ax1_5.grid(which='both', axis='both')
    ax1_5.axvline(cutoff_frequency_slider.val, color='green')

    # Mettre à jour la fenêtre de l'influence du canal :
    canal_window.canvas.draw_idle()

    # Mettre à jour le signal d'un seul symbole pour le filtre de nyquist :
    rect_t_ny, rect_signal_ny = generate_rect_signal(duration, sampling_rate, 1/symbolic_rate, emplacement=int(symbolic_rate*duration/10)-1)
    rect_signal_ny_filtered = apply_nyquist_filter(rect_signal_ny, duration, sampling_rate, symbolic_rate, delta, rolloff_factor_slider.val)

    # Retracer les graphes :
    ax2_1.cla()
    line2_1 = draw_signal(rect_t_ny, rect_signal_ny, ax2_1, "Input signal", bit_rate_slider.val, duration/5, val=1.5)
    ax2_2.cla()
    line2_2 = draw_spectre(rect_signal_ny, ax2_2, "Spectrum of the input signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    ax2_3.cla()
    line2_3 = draw_signal(rect_t_ny, rect_signal_ny_filtered, ax2_3, "Filtered signal", bit_rate_slider.val, duration/5, val=1.5)
    ax2_4.cla()
    line2_4 = draw_spectre(rect_signal_ny_filtered, ax2_4, "Spectrum of the filtered signal", symbolic_rate, sampling_rate, bit_rate_slider.val)
    ax2_5.cla()
    ax2_6.cla()
    plot_nyquist(duration, bit_rate_slider.val, sampling_rate, delta, rolloff_factor_slider.val, ax2_5, ax2_6)

    # Mettre à jour la fenêtre du filtre de nyquist :
    nyquist_window.canvas.draw_idle()



# LES GLISSIERES :

# Glissière de débit :
bit_rate_slider_ax = homepage.add_axes([0.075, 0.15, 0.01, 0.75])
bit_rate_slider = Slider(
    ax=bit_rate_slider_ax,
    label='Binary\nrate [Bit/s]',
    valmin=1,
    valmax=8,
    valinit=bit_rate,
    valstep=1,
    orientation="vertical"
)
bit_rate_slider.on_changed(regenere)

# Glissière de l'ordre du filtre représentant le canal :
order_ax = canal_window.add_axes([0.025, 0.1, 0.0225, 0.75])
order_slider = Slider(
    ax=order_ax,
    label="Order",
    valmin=order,
    valmax=5*order,
    valinit=order,
    valstep = 1,
    orientation="vertical"
)
order_slider.on_changed(update)

# Glissière de la fréquence de coupure du filtre représentant le canal :
cutoff_frequency_ax = canal_window.add_axes([0.14, 0.1, 0.0225, 0.75])
cutoff_frequency_slider = Slider(
    ax=cutoff_frequency_ax,
    label='Cutoff\nfrequency [Hz]',
    valmin=0.25,
    valmax=15,
    valinit=cutoff_frequency,
    valstep=0.25,
    orientation="vertical"
)
cutoff_frequency_slider.on_changed(update)

# Glissière du rapport signal sur bruit du canal :
snr_ax = canal_window.add_axes([0.14, 0.025, 0.75, 0.0225])
snr_slider = Slider(
    ax=snr_ax,
    label='SNR [dB]',
    valmin=0,
    valmax=60,
    valinit=snr,
    valstep=1,
    orientation="horizontal"
)
snr_slider.on_changed(update)

# Glissière du facteur de roll-off :
rolloff_factor_ax = nyquist_window.add_axes([0.05, 0.1, 0.0225, 0.75])
rolloff_factor_slider = Slider(
    ax=rolloff_factor_ax,
    label='roll-off\nfactor',
    valmin=0,
    valmax=1,
    valinit=rolloff_factor,
    valstep=0.05,
    orientation="vertical"
)
rolloff_factor_slider.on_changed(update)



# LES BOUTONS :

# Bouton de l'oeil du signal de sortiee de canal :
def oeil_outcanal(event):
    ax_oeil_outcanal_window.cla()
    drawFullEyeDiagram(symbolic_signal_out, ax_oeil_outcanal_window, 1/bit_rate_slider.val, sampling_rate, "Eye pattern of the output signal")
    oeil_outcanal_window.canvas.draw_idle()
    oeil_outcanal_window.show()

oeil_outcanal_ax = homepage.add_axes([0.92,0.38,0.05,0.04])
button_oeil_outcanal = Button(oeil_outcanal_ax, 'Eye', hovercolor='0.975')
button_oeil_outcanal.on_clicked(oeil_outcanal)

# Bouton de l'oeil de nyquist :
def oeil_nyquist(event):
    ax_oeil_window.cla()
    drawFullEyeDiagram(filtered_signal, ax_oeil_window, 1/bit_rate_slider.val, sampling_rate, "Nyquist eye pattern")
    oeil_window.canvas.draw_idle()
    oeil_window.show()

oeil_ax = homepage.add_axes([0.92,0.175,0.05,0.04])
button_oeil = Button(oeil_ax, 'Eye', hovercolor='0.975')
button_oeil.on_clicked(oeil_nyquist)

# Bouton radio de codage :
def codage(label):
    global code_name
    code_name=label
    button_codage.labels[0].set_weight('normal')
    button_codage.labels[1].set_weight('normal')
    button_codage.labels[2].set_weight('normal')
    button_codage.labels[3].set_weight('normal')
    button_codage.labels[4].set_weight('normal')
    button_codage.labels[5].set_weight('normal')
    button_codage.labels[6].set_weight('normal')
    button_codage.labels[7].set_weight('normal')
    ptr = 0
    while list_code[ptr]!=code_name:
        ptr = ptr + 1
    button_codage.labels[ptr].set_weight('bold')
    update(label)
    homepage.canvas.draw()
codage_ax = homepage.add_axes([0.63,0.725,0.25,0.25])
codage_ax.set_facecolor('0.9')
button_codage = RadioButtons(
    codage_ax,
    list_code
)
button_codage.on_clicked(codage)
button_codage.labels[0].set_weight('bold')
for r in button_codage.labels: 
    r.set_fontsize(20*dsize)
    r.set_fontstyle('italic')
    
# Bouton de canal :
def canal(event) :
    canal_window.show()
canal_ax = homepage.add_axes([0.115,0.5,0.06,0.04])
button_canal = Button(canal_ax, 'Channel', hovercolor='0.975')
button_canal.on_clicked(canal)

# Bouton de nyquist :
def nyquist(event):
    nyquist_window.show()
nyquist_ax = homepage.add_axes([0.115,0.275,0.07,0.04])
button_nyquist = Button(
    nyquist_ax, 
    'Nyquist', 
    hovercolor='0.975'
)
button_nyquist.on_clicked(nyquist)

0

Animation de l'oeil de Nyquist :

In [6]:
from matplotlib import rc
%matplotlib qt

print("Signal à représenter :\n1 : Symbolic output signal\n2 : Nyquist-filtered signal")
choice=0
while(choice!=1 and choice!=2):
    choice = int(input("Faites votre choix (1 ou 2) : "))
    if choice==1 :
        signal = symbolic_signal_out
        print("Signal = Symbolic output signal\n")
    elif choice==2 :
        signal = filtered_signal
        print("Signal = Nyquist-filtered signal\n")
    else :
        print("Erreur ! Veuillez-recommencer")


delta=1/bit_rate_slider.val
ratio_bi_sy_rate = bit_rate_slider.val/symbolic_rate

fig=plt.figure("Animation of the eye pattern", figsize=(width_window/1.5,height_window/1.5))
axs = fig.subplots(2,1) #2 lignes et une colonne
plt.close()

text = str()
compteur = 0
a = np.zeros(int(duration*bit_rate_slider.val/(2/ratio_bi_sy_rate)))  # a = suite de symboles
for i in range(len(a)):
        a[i] = binary_signal[int(((sampling_rate/bit_rate_slider.val)/2)+(sampling_rate/bit_rate_slider.val)*i)]
j=0
while j<len(a)-1 : 
    if (compteur%10==0 and compteur!=0) :
            text+='...\n'
            compteur=0
    text += str(int(a[j])) + ' ' + str(int(a[j+1])) + ' | '
    j+=2
    compteur+=1

print("Binary sequence :\n")
print(text, '...')
print('\n')

text = str()
compteur = 0
a = np.zeros(int(duration*symbolic_rate/(2/ratio_bi_sy_rate)))  # a = suite de symboles
for i in range(len(a)):
        a[i] = symbolic_signal[int(((sampling_rate/symbolic_rate)/2)+(sampling_rate/symbolic_rate)*i)]
if (delta==1/symbolic_rate) :
    j=0
    while j<len(a)-1 : 
        if (compteur%10==0 and compteur!=0) :
             text+='...\n'
             compteur = 0
        text += str(int(a[j])) + ' ' + str(int(a[j+1])) + ' | '
        j+=2
        compteur+=1
else :
    j=0
    while j<len(a)-3 : 
        if (compteur%10==0 and compteur!=0) :
            text+='...\n'
            compteur = 0
        text += str(int(a[j])) + ' ' + str(int(a[j+1])) + ' ' + str(int(a[j+2])) + ' ' + str(int(a[j+3])) + ' | '
        j+=4
        compteur+=1

print("Symbolic sequence :\n")
print(text, '...')

t_eye1 = np.arange(0,2*delta,1/sampling_rate)
t_eye2 = np.arange(0,2*delta,1/sampling_rate)
axs[0].set_xlim(0,int(duration*ratio_bi_sy_rate/2))
axs[0].set_ylim(-2,2)
axs[0].set_title("Animation of the eye pattern", fontsize=10)
axs[0].set_ylabel('Amplitude', fontsize=8)
axs[0].xaxis.set_major_formatter(formatter(delta, chr(8710)))
axs[0].xaxis.set_major_locator(MultipleLocator(4*delta))
axs[0].xaxis.set_minor_locator(MultipleLocator(1*delta))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.1))
axs[0].grid(axis = 'y', linestyle='--', color='grey', linewidth=0.75)
for i in range(int(len(a))):
    axs[0].axvline(delta*i*ratio_bi_sy_rate, color='g', linestyle = '--', linewidth=0.75)

axs[1].set_xlim(0,2*delta)
axs[1].set_ylim(-2,2)
axs[1].set_ylabel('Amplitude', fontsize=8)
axs[1].xaxis.set_major_formatter(formatter(delta, chr(8710)))
axs[1].xaxis.set_major_locator(MultipleLocator(delta))
axs[1].xaxis.set_minor_locator(MultipleLocator(0.1*delta))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))
axs[1].grid(axis = 'both', linestyle='--', color='grey', linewidth=0.75)

lines = [[],[]]

for i in range(int(duration/(4*delta/ratio_bi_sy_rate))):
    t_eye2 = np.concatenate((t_eye2,t_eye1))

# Fonction d'animation :
def animate(frame):
    start_index = int((frame + 1) * 2 * delta * sampling_rate) + 1
    end_index = int((frame + 2) * 2 * delta * sampling_rate) - 1
    lines[0].append(axs[0].plot([], [])[0])  # Créer une nouvelle ligne et l'ajouter à la liste
    lines[1].append(axs[1].plot([], [])[0])
    lines[0][frame].set_data(t[start_index:end_index], signal[start_index:end_index])
    lines[1][frame].set_data(t_eye2[start_index:end_index], signal[start_index:end_index])
    return lines[0] + lines[1] # Retourner toutes les lignes

ani = animation.FuncAnimation(fig, animate, frames=int(duration/(4*delta/ratio_bi_sy_rate)), blit=True, interval=200, repeat=False)

rc('animation', html = 'jshtml')
ani

Signal à représenter :
1 : Symbolic output signal
2 : Nyquist-filtered signal
Signal = Symbolic output signal

Binary sequence :

1 1 | 0 0 | 0 1 | 1 1 | 1 0 | 1 1 | 1 0 | 0 0 | 1 1 | 1 1 | ...
0 1 | 1 0 | 1 0 | 0 1 | 0 0 | 0 0 | 1 1 | 0 1 | 1 0 | 1 0 | ...
0 0 | 1 0 | 1 1 | 1 0 | 0 0 |  ...


Symbolic sequence :

1 1 | -1 -1 | -1 1 | 1 1 | 1 -1 | 1 1 | 1 -1 | -1 -1 | 1 1 | 1 1 | ...
-1 1 | 1 -1 | 1 -1 | -1 1 | -1 -1 | -1 -1 | 1 1 | -1 1 | 1 -1 | 1 -1 | ...
-1 -1 | 1 -1 | 1 1 | 1 -1 | -1 -1 |  ...


Vérification SBPA :

In [10]:
bit_rate = 10
t, signal = generate_binary_signal(duration, sampling_rate, bit_rate)
a = np.zeros(int(duration*bit_rate))  # a = suite de symboles
t_ = np.zeros(len(a))
for i in range(len(a)):
    a[i] = signal[int(((sampling_rate/bit_rate)/2)+(sampling_rate/bit_rate)*i)]
    t_[i] = t[int(((sampling_rate/bit_rate)/2)+(sampling_rate/bit_rate)*i)]
    if a[i]==0 : a[i]=-1
delta=1/bit_rate
print(a)
Rxx=0
for i in range(int(len(a)*delta)) :
    Rxx+=a[i]*a[int(i-delta)]
    print(Rxx)
Rxx=Rxx/(len(a)*delta)
print('\n')
print('Rxx = ' + str(Rxx))
R = np.zeros(int(duration*bit_rate))
R[0] = np.sqrt((Rxx*len(a))**2)
R[-1] = R[0]
R[1:-2] = Rxx
Fig = plt.figure("Fonction d'auto-corrélation du signal binaire")
ax = Fig.add_subplot(111)
ax.set_title("Fonction d'auto-corrélation du signal binaire")
ax.plot(t_, R)
ax.set_xlabel("t [s]")
ax.set_ylabel("Rx(t)")
ax.set_xlim(-0.01*len(a)*delta,1.01*len(a)*delta)
ax.set_ylim(-10,R[0]+10)
ax.xaxis.set_major_locator(MultipleLocator(bit_rate*delta))
ax.yaxis.set_major_locator(MultipleLocator(bit_rate))
ax.grid(axis = 'both', linestyle='--', color='grey', linewidth=0.75)
Fig.show()

[-1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1.  1.  1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1.
  1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  1.  1.  1. -1. -1.  1. -1. -1.
  1. -1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1.  1.
 -1. -1. -1. -1. -1.  1.  1. -1.  1.  1. -1. -1. -1.  1. -1. -1.  1. -1.
  1.  1. -1. -1.  1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  1. -1. -1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1.  1. -1.  1.  1.  1.
 -1. -1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1. -1. -1. -1.  1. -1.  1.
  1. -1. -1.  1.  1. -1.  1.  1. -1. -1.  1.  1. -1.  1.  1. -1.  1.  1.
  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1. -1. -1.  1.  1. -1.  1.  1.
  1.  1. -1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.
  1.  1.]
1.0
0.0
1.0
0.0
-1.0
-2.0
-1.0
-2.0
-3.0
-4.0
-3.0
-2.0
-1.0
-2.0
-1.0
0.0
1.0
0.0
-1.0
0.0


Rxx = 0.0
